不做太多的数据分析，最后求不同模型的得分结果

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge # L2正则
from sklearn.neighbors import KNeighborsRegressor # KNN
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
# 将datetime 中小时取出，使用字段hour进行存储
df['hour'] = pd.to_datetime(df['datetime']).map(lambda x: x.hour)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4


In [4]:
del df['datetime']
del df['casual']
del df['registered']

In [5]:
y = df[['count']]
x = df.drop('count', axis=1)
x.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour
0,1,0,0,1,9.84,14.395,81,0.0,0
1,1,0,0,1,9.02,13.635,80,0.0,1
2,1,0,0,1,9.02,13.635,80,0.0,2
3,1,0,0,1,9.84,14.395,75,0.0,3
4,1,0,0,1,9.84,14.395,75,0.0,4


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

L2正则

In [8]:
l2 = Ridge()
pg = {'alpha': [0.1, 0.3, 0.5, 0.8, 1]}
model = GridSearchCV(l2, pg, cv=3)
model.fit(x_train, y_train)
bp = model.best_params_
# 重新创建L2模型，找到最优参数直接处理
l2 = Ridge(alpha = bp.get('alpha'))
l2.fit(x_train, y_train)
l2.score(x_test, y_test)

0.32963459130352935

KNN

In [9]:
knn = KNeighborsRegressor()
pg = {'n_neighbors': [3, 4, 5, 6]}
model = GridSearchCV(knn, pg, cv=3)
model.fit(x_train, y_train)
bp = model.best_params_
knn = KNeighborsRegressor(n_neighbors=bp.get('n_neighbors'))
knn.fit(x_train, y_train)
knn.score(x_test, y_test)

0.516776400888801

决策树

In [10]:
tree = DecisionTreeRegressor()
pg = {'max_depth': [3, 4, 5]}
model = GridSearchCV(tree, pg, cv=3)
model.fit(x_train, y_train)
bp = model.best_params_
tree = DecisionTreeRegressor(max_depth=bp.get('max_depth'))
tree.fit(x_train, y_train)
tree.score(x_test, y_test)

0.5943835465509103

随机森林

In [11]:
# n_estimators 由多少个决策树组成
# max_depth 最大深度
rf = RandomForestRegressor()
pg = {'n_estimators': [50, 80], 'max_depth': [3, 4, 5]}
model = GridSearchCV(rf, pg, cv=3)
model.fit(x_train, y_train)
bp = model.best_params_
rf = RandomForestRegressor(n_estimators=bp.get('n_estimators'), max_depth=bp.get('max_depth'))
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.6308603702739006